<a href="https://www.kaggle.com/code/ibkya12/map-0-98-leashmp-competition?scriptVersionId=180665022" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 45.4 MB/s eta 0:00:00


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 77.8 MB/s eta 0:00:00


## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

In [3]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'
test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 220000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 220000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,294685184,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Cl.Cl.NCc1nnc2n1CCOCC2,Cl.Nc1ncns1,[N-]=[N+]=NCCC[C@H](Nc1nc(NCc2nnc3n2CCOCC3)nc(...,sEH,0
1,268230221,O=C(O)C[C@H](Cc1ccc(Cl)cc1)NC(=O)OCC1c2ccccc2-...,COCCNC(=O)CN.Cl,Cc1cc(N)cc(Cl)c1,COCCNC(=O)CNc1nc(Nc2cc(C)cc(Cl)c2)nc(N[C@H](CC...,sEH,0
2,283326241,O=C(O)[C@@H]1C[C@@H]2CCCC[C@@H]2N1C(=O)OCC1c2c...,CC(F)(F)CN.Cl,NCc1cn2cccc(F)c2n1,CC(F)(F)CNc1nc(NCc2cn3cccc(F)c3n2)nc(N2[C@H](C...,HSA,0
3,3797260,C#CC[C@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cn1c(CN)n[nH]c1=O,NC/C=C/Br,C#CC[C@H](CC(=O)N[Dy])Nc1nc(NC/C=C/Br)nc(NCc2n...,HSA,0
4,261074099,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cn1ncc2cc(N)ccc21,Nc1nc2ccccc2o1,Cn1ncc2cc(Nc3nc(Nc4nc5ccccc5o4)nc(N[C@H](CC(=O...,sEH,0


## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

## Train Model

In [6]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,ecfp
0,294685184,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Cl.Cl.NCc1nnc2n1CCOCC2,Cl.Nc1ncns1,[N-]=[N+]=NCCC[C@H](Nc1nc(NCc2nnc3n2CCOCC3)nc(...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7d8ab5be4a50>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,268230221,O=C(O)C[C@H](Cc1ccc(Cl)cc1)NC(=O)OCC1c2ccccc2-...,COCCNC(=O)CN.Cl,Cc1cc(N)cc(Cl)c1,COCCNC(=O)CNc1nc(Nc2cc(C)cc(Cl)c2)nc(N[C@H](CC...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7d8ab5a314d0>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
2,283326241,O=C(O)[C@@H]1C[C@@H]2CCCC[C@@H]2N1C(=O)OCC1c2c...,CC(F)(F)CN.Cl,NCc1cn2cccc(F)c2n1,CC(F)(F)CNc1nc(NCc2cn3cccc(F)c3n2)nc(N2[C@H](C...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7d8ab5a31540>,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ..."
3,3797260,C#CC[C@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cn1c(CN)n[nH]c1=O,NC/C=C/Br,C#CC[C@H](CC(=O)N[Dy])Nc1nc(NC/C=C/Br)nc(NCc2n...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7d8ab5a315b0>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,261074099,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cn1ncc2cc(N)ccc21,Nc1nc2ccccc2o1,Cn1ncc2cc(Nc3nc(Nc4nc5ccccc5o4)nc(N[C@H](CC(=O...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7d8ab5a31620>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


In [8]:
from xgboost import XGBClassifier

In [9]:
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train , y_train)

y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.97


# Best Score received with CatBoostClassifier

In [10]:
from catboost import CatBoostClassifier
catb_model = CatBoostClassifier().fit(X_train, y_train)
y_pred_proba = catb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Learning rate set to 0.125982
0:	learn: 0.6497046	total: 236ms	remaining: 3m 55s
1:	learn: 0.6137035	total: 433ms	remaining: 3m 35s
2:	learn: 0.5905922	total: 596ms	remaining: 3m 17s
3:	learn: 0.5694919	total: 784ms	remaining: 3m 15s
4:	learn: 0.5508179	total: 975ms	remaining: 3m 13s
5:	learn: 0.5362839	total: 1.16s	remaining: 3m 12s
6:	learn: 0.5241346	total: 1.33s	remaining: 3m 9s
7:	learn: 0.5146113	total: 1.5s	remaining: 3m 5s
8:	learn: 0.5081442	total: 1.65s	remaining: 3m 1s
9:	learn: 0.5017184	total: 1.79s	remaining: 2m 56s
10:	learn: 0.4945165	total: 1.95s	remaining: 2m 55s
11:	learn: 0.4880964	total: 2.12s	remaining: 2m 54s
12:	learn: 0.4802098	total: 2.31s	remaining: 2m 55s
13:	learn: 0.4743435	total: 2.48s	remaining: 2m 54s
14:	learn: 0.4707929	total: 2.62s	remaining: 2m 52s
15:	learn: 0.4662524	total: 2.84s	remaining: 2m 54s
16:	learn: 0.4626531	total: 3.03s	remaining: 2m 55s
17:	learn: 0.4585043	total: 3.28s	remaining: 2m 58s
18:	learn: 0.4557151	total: 3.49s	remaining: 3m


Look at that Average Precision score. We did amazing! 

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [11]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-predict-chemical-bindings/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=150000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = catb_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
